In [1]:
import random
import numpy as np
import torch

# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# define input sequence
in_seq1 = array([x for x in range(0,100,10)])
in_seq2 = array([x for x in range(5,105,10)])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

In [3]:
in_seq2

array([[ 5],
       [15],
       [25],
       [35],
       [45],
       [55],
       [65],
       [75],
       [85],
       [95]])

In [6]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 20 # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        cell_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):        
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x,self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [7]:
n_features = 2 # this is number of parallel inputs
n_timesteps = 3 # this is number of timesteps

# convert dataset into input/output
X, y = split_sequences(dataset, n_timesteps)
print(X.shape, y.shape)

# create NN
mv_net = MV_LSTM(n_features,n_timesteps)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-1)

train_episodes = 500
batch_size = 16


(8, 3, 2) (8,)


In [34]:
mv_net.train()
for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]    
        
        x_batch = torch.tensor(inpt,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)
    
        mv_net.init_hidden(x_batch.size(0))
        print(x_batch.size(0))
        break
    #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
    #    lstm_out.contiguous().view(x_batch.size(0),-1)
        output = mv_net(x_batch) 
        loss = criterion(output.view(-1), y_batch)  
        
        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 
    print('step : ' , t , 'loss : ' , loss.item())

8
step :  0 loss :  0.5828284621238708
8
step :  1 loss :  0.5828284621238708
8
step :  2 loss :  0.5828284621238708
8
step :  3 loss :  0.5828284621238708
8
step :  4 loss :  0.5828284621238708
8
step :  5 loss :  0.5828284621238708
8
step :  6 loss :  0.5828284621238708
8
step :  7 loss :  0.5828284621238708
8
step :  8 loss :  0.5828284621238708
8
step :  9 loss :  0.5828284621238708
8
step :  10 loss :  0.5828284621238708
8
step :  11 loss :  0.5828284621238708
8
step :  12 loss :  0.5828284621238708
8
step :  13 loss :  0.5828284621238708
8
step :  14 loss :  0.5828284621238708
8
step :  15 loss :  0.5828284621238708
8
step :  16 loss :  0.5828284621238708
8
step :  17 loss :  0.5828284621238708
8
step :  18 loss :  0.5828284621238708
8
step :  19 loss :  0.5828284621238708
8
step :  20 loss :  0.5828284621238708
8
step :  21 loss :  0.5828284621238708
8
step :  22 loss :  0.5828284621238708
8
step :  23 loss :  0.5828284621238708
8
step :  24 loss :  0.5828284621238708
8
step :  

8
step :  425 loss :  0.5828284621238708
8
step :  426 loss :  0.5828284621238708
8
step :  427 loss :  0.5828284621238708
8
step :  428 loss :  0.5828284621238708
8
step :  429 loss :  0.5828284621238708
8
step :  430 loss :  0.5828284621238708
8
step :  431 loss :  0.5828284621238708
8
step :  432 loss :  0.5828284621238708
8
step :  433 loss :  0.5828284621238708
8
step :  434 loss :  0.5828284621238708
8
step :  435 loss :  0.5828284621238708
8
step :  436 loss :  0.5828284621238708
8
step :  437 loss :  0.5828284621238708
8
step :  438 loss :  0.5828284621238708
8
step :  439 loss :  0.5828284621238708
8
step :  440 loss :  0.5828284621238708
8
step :  441 loss :  0.5828284621238708
8
step :  442 loss :  0.5828284621238708
8
step :  443 loss :  0.5828284621238708
8
step :  444 loss :  0.5828284621238708
8
step :  445 loss :  0.5828284621238708
8
step :  446 loss :  0.5828284621238708
8
step :  447 loss :  0.5828284621238708
8
step :  448 loss :  0.5828284621238708
8
step :  449 lo

In [35]:
inpt

array([[[ 0,  5],
        [10, 15],
        [20, 25]],

       [[10, 15],
        [20, 25],
        [30, 35]],

       [[20, 25],
        [30, 35],
        [40, 45]],

       [[30, 35],
        [40, 45],
        [50, 55]],

       [[40, 45],
        [50, 55],
        [60, 65]],

       [[50, 55],
        [60, 65],
        [70, 75]],

       [[60, 65],
        [70, 75],
        [80, 85]],

       [[70, 75],
        [80, 85],
        [90, 95]]])

In [11]:
mv_net(])

AttributeError: 'list' object has no attribute 'size'

In [69]:
tmp = [[[10,15],[20,20],[30,35]]]

In [70]:
ex = torch.tensor(tmp, dtype=torch.float32)

In [71]:
mv_net.init_hidden(ex.size(0))

In [72]:
mv_net(ex)

tensor([[66.5749]], grad_fn=<AddmmBackward>)

In [68]:
ex

tensor([[[10.,  5.],
         [20., 10.],
         [30., 15.]]])

## ex

In [26]:
ex.size()

torch.Size([1, 3, 2])

In [24]:
ex

tensor([[10, 11],
        [11, 12],
        [11, 13]])

In [20]:
inpt

array([[[ 0,  5],
        [10, 15],
        [20, 25]],

       [[10, 15],
        [20, 25],
        [30, 35]],

       [[20, 25],
        [30, 35],
        [40, 45]],

       [[30, 35],
        [40, 45],
        [50, 55]],

       [[40, 45],
        [50, 55],
        [60, 65]],

       [[50, 55],
        [60, 65],
        [70, 75]],

       [[60, 65],
        [70, 75],
        [80, 85]],

       [[70, 75],
        [80, 85],
        [90, 95]]])

In [36]:
ex = torch.tensor(tmp, dtype=torch.float32)

In [38]:
ex.size(0)

1

In [39]:
mv_net.init_hidden(1)

In [40]:
mv_net(ex)

tensor([[66.7826]], grad_fn=<AddmmBackward>)

tensor([[[10., 11.],
         [11., 12.],
         [11., 13.]]])